In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs 
import requests
import lxml

req = requests.get("https://www.espncricinfo.com/records/season/team-match-results/2024-2024?trophy=89")
soup = bs(req.content)

## Converting Table to DataFrame:

In [ ]:
# All matches summary with match_id
table = soup.find('table')
df = pd.read_html(str(table))[0]
df

## Match Summary DataFrame

In [ ]:
df['Match Date'] = pd.to_datetime(df['Match Date'])
df['Match Date'] = df['Match Date'].dt.strftime('%d-%m-%Y')
df.tail()

In [ ]:
df.rename(columns={'Scorecard': 'match_id'}, inplace=True)
df.rename(columns={'Match Date': 'match_date'}, inplace=True)
df.rename(columns={'Ground': 'ground'}, inplace=True)
df.rename(columns={'Margin': 'margin'}, inplace=True)
df.rename(columns={'Winner': 'winner'}, inplace=True)
df.rename(columns={'Team 2': 'team2'}, inplace=True)
df.rename(columns={'Team 1': 'team1'}, inplace=True)
df.head()

## Getting Individual Match Scorecards

In [ ]:
print(table.prettify())

In [ ]:
links = table.find_all('a')
links = links[1::2]
links = [link['href'] for link in links]  
links

## Scraping scorecard for one Match :

In [ ]:
links[0]
link = "https://www.espncricinfo.com" + links[0]  
soup1 = bs(requests.get(link).content)

match = soup1.find_all('table')
bat_innings1 = match[0]
bowl_innings1 = match[1]
bat_innings2 = match[2]
bowl_innings2 = match[3]

### For batting innings 1

In [ ]:
df = pd.read_html(str(bat_innings1))[0]
df.drop(columns=['Unnamed: 8', 'Unnamed: 9', 'M'], inplace=True)
df.dropna(axis=0, inplace=True)

# Renaming the columns
df.rename(columns={'Unnamed: 1': 'out or not_out'}, inplace=True)

# Creating an out or not_out column
df['out or not_out'] = df['out or not_out'].apply(lambda x: 'not_out' if 'not' in x else 'out')
df.dropna(inplace=True)

df['6s'] = pd.to_numeric(df['6s'], errors='coerce')
df.dropna(subset=['6s'], inplace=True)
df

### For bowling innings 1

In [ ]:
df2 = pd.read_html(str(bowl_innings1))[0]

# Converting R columns to integer and dropping the rows with non numeric values
df2['R'] = pd.to_numeric(df2['R'], errors='coerce')
df2.dropna(subset=['R'], inplace=True)
df2

### For batting innings 2

In [ ]:
df3 = pd.read_html(str(bat_innings2))[0]
df3.drop(columns=['Unnamed: 8', 'Unnamed: 9', 'M'], inplace=True)
df3.dropna(axis=0, inplace=True)

# Creating an out or not_out column
df3['out or not_out'] = df3['out or not_out'].apply(lambda x: 'not_out' if 'not' in x else 'out')
df3.dropna(inplace=True)

df3['sixes'] = pd.to_numeric(df3['sixes'], errors='coerce')
df3.dropna(subset=['sixes'], inplace=True)
df3

### For bowling innings 2

In [ ]:
df4 = pd.read_html(str(bowl_innings2))[0]

# Converting R columns to integer and dropping the rows with non numeric values
df4['R'] = pd.to_numeric(df4['R'], errors='coerce')
df4.dropna(subset=['R'], inplace=True)

df4

## Implementing the above codes for list of Links

In [ ]:
links

,Bowling,O,M,R,W,ECON,0s,4s,6s,WD,NB
0,Arshdeep Singh,4,0,20.0,2,5.00,12,2,0,0,0
2,Jasprit Bumrah,4,0,18.0,2,4.50,14,2,0,0,0
4,Axar Patel,4,0,49.0,1,12.25,8,5,3,2,0
6,Kuldeep Yadav,4,0,45.0,0,11.25,6,3,3,0,0
7,Hardik Pandya,3,0,20.0,3,6.66,9,1,1,1,1
9,Ravindra Jadeja,1,0,12.0,0,12.00,0,0,1,0,0
